In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Load CSV
df = pd.read_csv("/mnt/data/Nat_Gas.csv")

# Ensure proper column names
df.columns = ["date", "price"]

# Convert to datetime
df["date"] = pd.to_datetime(df["date"])

# Sort chronologically
df = df.sort_values("date").reset_index(drop=True)

df.head()


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df["date"], df["price"], marker="o")
plt.title("Monthly Natural Gas Prices (End-of-Month)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(True)
plt.show()


In [ ]:
df["month"] = df["date"].dt.month

monthly_avg = df.groupby("month")["price"].mean()
overall_avg = df["price"].mean()

seasonality = monthly_avg - overall_avg


In [ ]:
df["deseasonal_price"] = df.apply(
    lambda row: row["price"] - seasonality[row["month"]],
    axis=1
)


In [ ]:
start_date = df["date"].min()
df["t"] = (df["date"] - start_date).dt.days

trend_function = interp1d(
    df["t"],
    df["deseasonal_price"],
    kind="linear",
    fill_value="extrapolate"
)


In [ ]:
def estimate_gas_price(input_date):
    """
    Returns estimated natural gas price for any past date
    and extrapolates up to 1 year into the future.
    """
    if isinstance(input_date, str):
        input_date = pd.to_datetime(input_date)

    t = (input_date - start_date).days
    trend_price = float(trend_function(t))
    seasonal_adj = seasonality[input_date.month]

    return round(trend_price + seasonal_adj, 3)


In [ ]:
future_dates = pd.date_range(
    start=df["date"].max() + pd.offsets.MonthEnd(1),
    end=df["date"].max() + pd.DateOffset(years=1),
    freq="M"
)

future_prices = [estimate_gas_price(d) for d in future_dates]

future_df = pd.DataFrame({
    "date": future_dates,
    "price": future_prices
})


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df["date"], df["price"], label="Historical", marker="o")
plt.plot(future_df["date"], future_df["price"], 
         label="1-Year Extrapolation", linestyle="--")

plt.title("Natural Gas Priceං Curve with Seasonality & Extrapolation")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print("Price on 2021-12-15:", estimate_gas_price("2021-12-15"))
print("Price on 2024-06-10:", estimate_gas_price("2024-06-10"))
print("Price on 2025-03-15:", estimate_gas_price("2025-03-15"))
